In [36]:
!pip install scapy
!pip install hdrhistogram
!pip install tzlocal

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [10]:
from scapy.all import rdpcap

print(f"This will take some time... Python PCAP library is slow...")
file_path = '/mnt/e/TecVal-Latency-Captures/fix-traffic-on-port-10001-7.5k.pcap'   #fix-traffic-port10001.pcap
packets = rdpcap(file_path)
print(f"Total packets found: {len(packets)}")

This will take some time... Python PCAP library is slow...
Total packets found: 422076


In [11]:
import re
from datetime import datetime, timezone, timedelta
from hdrh.histogram import HdrHistogram
import pytz
import tzlocal

# Function to extract FIX tags from a message
def extract_fix_tags(fix_message):
    tags = {}
    # Split the message by SOH (Start of Header) character
    fields = fix_message.split('\x01')
    for field in fields:
        if '=' in field:
            tag, value = field.split('=', 1)
            tags[tag] = value
    return tags

# Function to convert FIX UTCTimestamp to a Python datetime object
def convert_to_datetime(fix_timestamp):
    print(f"extracting timestamp from " + fix_timestamp)
    try:
        return datetime.strptime(fix_timestamp, "%Y%m%d-%H:%M:%S.%f")
    except ValueError:
        return datetime.strptime(fix_timestamp, "%Y%m%d-%H:%M:%S")

def delta_nanos (epoch_seconds1, nanoseconds1, epoch_seconds2, nanoseconds2):
    delta_seconds = epoch_seconds1 - epoch_seconds2
    delta_nanos = nanoseconds1 - nanoseconds2
    return delta_seconds*1000000000 + delta_nanos

# Converts date and time specified in UTCTimestamp format "YYYYMMDD-HH:MM:SS.nnnnnnnnn" to epoch seconds and nanoseconds
def fix_timestamp_to_epoch_nano(utctimestamp):
    timestamp_parts = utctimestamp.split('.')
    timestamp_without_nano = timestamp_parts[0]
    nanoseconds = int(timestamp_parts[1]) if len(timestamp_parts) > 1 else 0
    epoch_seconds = int(datetime.strptime(timestamp_without_nano, "%Y%m%d-%H:%M:%S").timestamp())
    return (epoch_seconds, nanoseconds)

def get_utc_offset_seconds():
    utc_now = datetime.now(pytz.utc)
    local_tz = tzlocal.get_localzone()
    local_now = datetime.now(local_tz)

    return int(local_tz.utcoffset(local_now).total_seconds())

UTC_OFFSET_SECONDS = get_utc_offset_seconds()

def packet_time(packet_timestamp):
    seconds = int(packet_timestamp)
    fractional_part = packet_timestamp - seconds
    nanoseconds = int(fractional_part*1000000000)
    #print(f"{packet_timestamp} => seconds: {seconds}, nanos: {nanoseconds}")
    return (seconds - UTC_OFFSET_SECONDS, nanoseconds)

In [12]:
##################################################################################################
# from TransactTime(60) until SendingTime(52)
##################################################################################################
histogram = HdrHistogram(1, 10**9, significant_figures=3)  # From 1 ns to 1 sec in ns

for packet in packets:
    if packet.haslayer('Raw'):
        raw_data = packet['Raw'].load.decode('ascii', errors='ignore')
        if raw_data.startswith("8=FIXT"):
            fix_tags = extract_fix_tags(raw_data)
            if '52' in fix_tags and '60' in fix_tags:
                sending_time = fix_tags['52']
                sending_time_seconds,sending_time_nanos = fix_timestamp_to_epoch_nano(sending_time)
                transact_time = fix_tags['60']
                transact_time_seconds,transact_time_nanos = fix_timestamp_to_epoch_nano(transact_time)
                packet_time_seconds,packet_time_nanos = packet_time(packet.time)
                timestamp_delta = delta_nanos(sending_time_seconds, sending_time_nanos, transact_time_seconds, transact_time_nanos)
                histogram.record_value(timestamp_delta)

#TODO:
NANOS_IN_MILLI = 1_000_000

# Fetch latency values
total_count = histogram.get_total_count()
latencies = {
    'P0': histogram.get_value_at_percentile(0),
    'P25': histogram.get_value_at_percentile(25),
    'P50': histogram.get_value_at_percentile(50),
    'P90': histogram.get_value_at_percentile(90),
    'P99': histogram.get_value_at_percentile(99),
    'P999': histogram.get_value_at_percentile(99.9),
    'P9999': histogram.get_value_at_percentile(99.99),
    'P99999': histogram.get_value_at_percentile(99.999),
    'P100': histogram.get_value_at_percentile(100)
}

# Convert nanoseconds to milliseconds
latencies_ms = {k: v / NANOS_IN_MILLI for k, v in latencies.items()}

# Output results
print(f"Total FIX messages processed: {total_count}")
for percentile, latency in latencies_ms.items():
    print(f"{percentile:<6} Latency: {latency:.2f} ms")

Total FIX messages processed: 282962
P0     Latency: 0.36 ms
P25    Latency: 0.89 ms
P50    Latency: 1.10 ms
P90    Latency: 1.49 ms
P99    Latency: 1.67 ms
P999   Latency: 1.76 ms
P9999  Latency: 2.66 ms
P99999 Latency: 4.17 ms
P100   Latency: 4.53 ms


In [13]:
from decimal import Decimal, getcontext
getcontext().prec = 30

##################################################################################################
# from TransactTime(60) until PCAP Packet Time
##################################################################################################
histogram = HdrHistogram(1, 10**9, significant_figures=3)  # From 1 ns to 1 sec in ns

for packet in packets:
    if packet.haslayer('Raw'):
        raw_data = packet['Raw'].load.decode('ascii', errors='ignore')
        if raw_data.startswith("8=FIXT"):
            fix_tags = extract_fix_tags(raw_data)
            if '52' in fix_tags and '60' in fix_tags: # not really using tag 52
                pcap_time_seconds,pcap_time_nanos = packet_time(packet.time)
                transact_time = fix_tags['60']
                transact_time_seconds,transact_time_nanos = fix_timestamp_to_epoch_nano(transact_time)
                
                timestamp_delta = delta_nanos(pcap_time_seconds, pcap_time_nanos, transact_time_seconds, transact_time_nanos)
                histogram.record_value(timestamp_delta)


#TODO:
NANOS_IN_MILLI = 1_000_000

# Fetch latency values
total_count = histogram.get_total_count()
latencies = {
    'P0': histogram.get_value_at_percentile(0),
    'P25': histogram.get_value_at_percentile(25),
    'P50': histogram.get_value_at_percentile(50),
    'P90': histogram.get_value_at_percentile(90),
    'P99': histogram.get_value_at_percentile(99),
    'P999': histogram.get_value_at_percentile(99.9),
    'P9999': histogram.get_value_at_percentile(99.99),
    'P99999': histogram.get_value_at_percentile(99.999),
    'P100': histogram.get_value_at_percentile(100)
}

# Convert nanoseconds to milliseconds
latencies_ms = {k: v / NANOS_IN_MILLI for k, v in latencies.items()}

# Output results
print(f"Total FIX messages processed: {total_count}")
for percentile, latency in latencies_ms.items():
    print(f"{percentile:<6} Latency: {latency:.2f} ms")

Total FIX messages processed: 282962
P0     Latency: 0.53 ms
P25    Latency: 1.21 ms
P50    Latency: 1.41 ms
P90    Latency: 1.79 ms
P99    Latency: 2.04 ms
P999   Latency: 2.18 ms
P9999  Latency: 3.36 ms
P99999 Latency: 4.80 ms
P100   Latency: 5.03 ms
